# Test adding of Streamlines, Text, Logo, Spheres

In [ ]:
from pkg_resources import resource_filename

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import astro3d
from astro3d import image_stack

cyan = image_stack.BaseCyan

## Create stack

In [ ]:
dpi_x = 600
dpi_y = 300
dz    = 27e-4
L     = 4.

input = np.ones(np.array([dpi_x * L / 2.54, dpi_y * L / 2.54, L / dz, 3], dtype=int), dtype=np.uint8) * 255

stack = astro3d.image_stack.IStack(input)

### Add a box

In [ ]:
stack.add_box(pos=L/4, length=L/2)

### Add a blue sphere

In [ ]:
stack.add_sphere(pos=L / 2 * np.ones(3), radius=L/10, color=cyan)

### Add some text

In [ ]:
opts = dict(height=L/15, width=None, depth=L/40, col=cyan)

stack.add_text('b-l-xz', [0.1 * L, 0.02 * L, 0.05 * L], **opts, plane='xz')
stack.add_text('b-r-xz', [0.7 * L, 0.02 * L, 0.05 * L], **opts, plane='xz')
stack.add_text('t-l-xz', [0.1 * L, 0.02 * L, 0.90 * L], **opts, plane='xz')
stack.add_text('t-r-xz', [0.7 * L, 0.02 * L, 0.90 * L], **opts, plane='xz')

stack.add_text('b-l-xy', [0.05 * L, 0.05 * L, 0.05 * L], **opts, plane='xy')
stack.add_text('b-r-xy', [0.75 * L, 0.05 * L, 0.05 * L], **opts, plane='xy')
stack.add_text('t-l-xy', [0.05 * L, 0.90 * L, 0.05 * L], **opts, plane='xy')
stack.add_text('t-r-xy', [0.75 * L, 0.90 * L, 0.05 * L], **opts, plane='xy')

stack.add_text('b-l-yz', [0.02 * L, 0.1 * L, 0.05 * L], **opts, plane='yz')
stack.add_text('b-r-yz', [0.02 * L, 0.7 * L, 0.05 * L], **opts, plane='yz')
stack.add_text('t-l-yz', [0.02 * L, 0.1 * L, 0.90 * L], **opts, plane='yz')
stack.add_text('t-r-yz', [0.02 * L, 0.7 * L, 0.90 * L], **opts, plane='yz')

### Add a logo

this image has no alpha, so white would become white, but we want white material be used for dithering
the right color. What's white in the image should become transparent. So we need to define an alpha map

In [ ]:
im = np.array(Image.open(resource_filename('astro3d', '/../figures/Logo_Origins+qr.png')).convert('RGBA'))
im[:, :, 3] = 255 * (np.array(im).mean(-1)<180)

In [ ]:
f, ax = plt.subplots()
f.set_facecolor('0.7')
ax.set_facecolor('0.7')
#ax.imshow(im[:,:,:3].mean(-1)<180)
ax.imshow(im)

#### Method 1: useing dithering

In [ ]:
stack.add_logo(im, [0.25 * L, 0.1 * L, 0.25 * L], 0.5 * L, L / 30, plane='xz')

stack.show_colors()
stack.empty_indices = [0]

f, ax = stack.show_view('xz', bg=[60, 60, 60])
ax.set_xlim(400, 600)
ax.set_ylim(400, 600)

#### Method 2: useing the threshold-method

In [ ]:
stack.add_logo(im, [0.25 * L, 0.1 * L, 0.25 * L], 0.5 * L, L / 30, plane='xz',
               threshold=0.2, pal=[[255, 255, 255],image_stack.BaseCyan])

f, ax = stack.show_view('xz', bg=[60, 60, 60])
ax.set_xlim(200, 400)
ax.set_ylim(400, 600)

## Display results

In [ ]:
f, ax = stack.three_views()
f.dpi = 200

## Compare stack slice and output image

In [ ]:
f, ax = plt.subplots()
ax.pcolormesh(stack._x, stack._y, stack.imgs.mean(-1)[:, :, 76].T)

In [ ]:
stack.save_images('test_stack')

In [ ]:
stack2 = image_stack.IStack('test_stack', dpi_x=600, dpi_y=300, dz=27e-4)

Check if the created stack and the read-in stack are identical:

In [ ]:
stack.imgs.shape == stack2.imgs.shape

In [ ]:
f, ax = plt.subplots()
ax.pcolormesh(stack2._x, stack2._y, stack2.imgs.mean(-1)[:, :, 76].T)

Display read-in stack

In [ ]:
stack2.show_colors()
stack2.empty_indices = [0]
stack2.three_views()